# Stellenausschreibung im Ausland (Viajob.ro)

Der Bundesrat setzt die Masseneinwanderungsinitiative in Etappen um. Seit dem 1.Juni 2018 gilt in der Schweiz die Stellenmeldepflicht. Sie wird in einem ersten Schritt in denjenigen Berufsarten eingeführt, in denen die gesamtschweizerische Arbeitslosenquote 8 Prozent erreicht oder überschreitet. Per 1. Januar 2020 wird dieser Schwellenwert auf 5 Prozent gesenkt. Trotz dem Ja zur Masseneinwanderungsinitiative schreiben Schweizer Firmen Jobs in EU-Ländern aus, zum Teil zu Dumpinglöhnen. Auf dem rumänischen Portal Viajob wird etwa ein Veterinär in Zürich gesucht für einen Lohn von 4000 CHF pro Monat. Mich interessiert in welchen Ländern, auf welchen Protalen, werden welche Jobs in der Schweiz, von wem und zu welchem Lohn ausgeschrieben. 

Quelle: https://www.viajob.ro/   

Ziel: Eine Tabelle mit allen Ausschreibungen für eine Stelle in der Schweiz. Zu jeder ausgeschriebenen Stelle soll der Beruf, Ort und Monatsgehalt angegeben werden. 

### Schritt 1: Import der Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Schritt 2: Webseite analysieren

Die Stellenangebote für die Schweiz erscheinen auf einer Unterseite. Schweiz auf rumänisch heisst "Elvetia". Alle gesuchten Informationen (Beruf, Ort, Gehalt) sind bereits dort verfügbar.  

https://www.viajob.ro/munca-strainatate?tara=Elvetia  

Zusätzliche Informationen gibt es pro Stellengesuch auf einer Unterwebseite. Die Url setzt sich in der Regel aus der Jobbezeichnung und Ort zusammen:

https://www.viajob.ro/locuri-de-munca-operator-cnc-geneva.html

In [3]:
url = 'https://www.viajob.ro/munca-strainatate?tara=Elvetia'
content = requests.get(url).text 
soup = BeautifulSoup(content, 'html.parser')
# soup

In [7]:
# Ort und Beruf auslesen

# 3 Varianten
# location_job = soup.select('h2.h2blockhome')
# location_job = soup.find_all('h2', class_='h2blockhome')
location_job = soup.find_all('h2', {'class':'h2blockhome'})
location_job[1]
location_job[1].text

'operator cnc\nGeneva'

**TIPP:** Webseite analysieren mit SelectorGadget. ACHTUNG funktioniert nur in Google Chrome!

**FRAGE:** Gibt es eine Möglichkeiten bereits hier Beruf und Job hier zu trennen und als separate Listen auszulesen? **JA!** Beruf und Ort werden durch \n voneinander getrennt und lässt sich dort splitten. Siehe unter Schritt 3 die verschiedenen Varianten wie das zB im For-Loop genutzt werden kann!

In [114]:
len(location_job)

45

In [9]:
salery = soup.find_all('strong', {'class':'salarluna'})
salery[1]
salery[1].text

'3900 CHF'

In [113]:
len(salery)

45

In [107]:
# BESSER: den ganzen Block auslesen
# ad = soup.select('div.blockprincipal')
ad = soup.find_all('div', class_="blockprincipal")
ad[0]

<div class="blockprincipal">
<img class="flag" src="https://www.viajob.ro/wp-content/uploads/2015/09/Switzerland.png"/><br/>
<a href="https://www.viajob.ro/locuri-de-munca-menaj-geneva.html" target="_blank"><img alt="locuri de munca menaj Geneva" class="blockdiv" height="180px" src="https://www.viajob.ro/wp-content/uploads/formidable/20/Locuri-de-munca-menaj-Geneva-250x173.jpg"/></a><br/>
<a href="https://www.viajob.ro/locuri-de-munca-menaj-geneva.html" target="_blank"></a></div>

In [109]:
ad[1]
ad[1].text

'\n\n\n'

**ACHTUNG:** Es gibt ein Problem. In diesem Block sind die h2 und strong nicht enthalten.

HTML-Code der ganzen Webseite https://www.viajob.ro/munca-strainatate?tara=Elvetia als txt-File den Output von soup abspeichern mittels Edit > Copy Cells > Copy Paste in html.txt

**GRUND:** Infos im Block wird nachgeladen mit Java-Script. Deshalb erscheint der Block leer. Wenn man mit Beautifulsoup das holen will ist es leer. Auf die Spitze getrieben wird das zB von Facebook.

**ABER** warum funktioniert es wenn ich eine Liste bauen für Ort, Beruf und Monatslohn? Könnte es zb oben auch mehrere blockprincipal-Blöcke geben? Analyse HTML-Code zeigt: die einzigen blockprincipal vorher sind im Stylguide drin.

In [50]:
# HTML-Code abspeichern

html_code = content    # Variable content wurde ganz am Anfang definiert 
 
with open('html_code_viajob.txt', 'w') as file:   
    file.write(html_code)    
    file.close()

In [ ]:
#Variante 2, um HTML abzuspeichern

#with open(“output1.html”, “w”) as file:
   #file.write(str(soup))

### Schritt 3: For-Loop

Es braucht einen Work around. Zum Glück sind alle Listen gleich lang, und die einzelnen Elemente korrepsondieren miteinander. Neue Liste bauen

In [37]:
# VARIANTE 1

lst = []
for description,chf in zip(location_job,salery):
    gesplittet = description.text.split('\n')
    mini_dict = {"Beruf": gesplittet[0],
                'Ort':gesplittet[1],
                 'Monatslohn':chf.text}
    
    lst.append(mini_dict)
#lst

In [38]:
# VARIANTE 2

lst = []
for description,chf in zip(location_job,salery):
    beruf, ort  = description.text.split('\n')
    mini_dict = {"Beruf": beruf,
                'Ort':ort,
                 'Monatslohn':chf.text}
    
    lst.append(mini_dict)
#lst

In [34]:
# VARIANTE 3

output = soup.find_all('h2', {'class':'h2blockhome'})
output2 = soup.find_all('strong', {'class':'salarluna'})

job = [item.text.split('\n')[0] for item in output]
location = [item.text.split('\n')[1] for item in output]
salary = [item.text for item in output2]

data = list(zip(job, location,salary))
df = pd.DataFrame.from_records(data,columns=["job","loc","salary"])
df.head(4)

,job,loc,salary
0,menaj,Geneva,3400 CHF
1,operator cnc,Geneva,3900 CHF
2,electrician intretinere,Geneva,3600 CHF
3,curatenie,Geneva,3400 CHF


### Schritt 4: Daten mit Pandas analysieren

In [49]:
df = pd.DataFrame(lst)
df.to_csv('list_open_positions_viajob.csv')

In [48]:
df.head(3)

,Beruf,Monatslohn,Ort
0,menaj,3400 CHF,Geneva
1,operator cnc,3900 CHF,Geneva
2,electrician intretinere,3600 CHF,Geneva
